# **Project Overview- Deep Q-Networks**

**Objective**: Implement a Deep Q-Network (DQN) to solve the Taxi-v3 problem using OpenAI Gym.

**Table of Contents**

Setup and Environment

* Install necessary Python packages including Gym and TensorFlow.
* Initialize the Taxi-v3 environment in OpenAI Gym.
* Define the state and action spaces for the DQN model.

**DQN Implementation**

* Construct the neural network architecture for the DQN agent.
* Implement the DQN algorithm for decision-making and learning.
* Train the DQN agent in the Taxi-v3 environment.

**Evaluation and Performance**

* Assess the DQN agent's learning progress over episodes.
* Evaluate the trained agent's performance in the environment.



In [ ]:
pip install gym numpy tensorflow


In [5]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def act(self, state):
        reshaped_state = np.reshape(state, [1, self.state_size])
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(reshaped_state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            reshaped_state = np.reshape(state, [1, self.state_size])
            reshaped_next_state = np.reshape(next_state, [1, self.state_size])

            target = reward if done else reward + self.gamma * np.amax(self.model.predict(reshaped_next_state)[0])
            target_f = self.model.predict(reshaped_state)
            target_f[0][action] = target
            self.model.fit(reshaped_state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

if __name__ == "__main__":
    env = gym.make('Taxi-v3')
    state_size = 1  # The Taxi-v3 environment uses a single integer for state representation
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)

    episodes = 100

    for e in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            if done:
                print(f"episode: {e+1}/{episodes}, total reward: {total_reward}, epsilon: {agent.epsilon:.2}")
                break

        if len(agent.memory) > 32:
            agent.replay(32)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step
episode: 62/100, total reward: -731, epsilon: 0.74
1/1 [==============================] - 0s 35ms/step
episode: 63/100, total reward: -614, epsilon: 0.73
1/1 [==============================] - 0s 51ms/step
episode: 64/100, total reward: -587, epsilon: 0.73
1/1 [==============================] - 0s 40ms/step
episode: 65/100, total reward: -560, epsilon: 0.73
1/1 [==============================] - 0s 31ms/step
episode: 66/100, total reward: -560, epsilon: 0.72
1/1 [==============================] - 0s 40ms/step
episode: 67/100, total reward: -569, epsilon: 0.72
1/1 [==============================] - 0s 66ms/step
episode: 68/100, total reward: -677, epsilon: 0.71
1/1 [==============================] - 0s 57ms/step
episode: 69/100, total reward: -632, epsilon: 0.71
1/1 [==============================] - 0s 31ms/step
episode: 70/100, total reward: -677, epsilon: 0.71
1/1 [=================

In [ ]:
average_score = total_evaluation_reward / evaluation_episodes
print(f"Average Score over {evaluation_episodes} Evaluation Episodes: {average_score}")



* **Score Interpretation**: In the Taxi-v3 game, the agent earns a reward for successfully dropping off a passenger and incurs a small time penalty for each time-step it takes to complete this task. Additional penalties are incurred for illegal pick-up and drop-off actions. The maximum score per episode is 20 (excluding the time penalty), obtained by perfectly picking up and dropping off a passenger in the shortest possible time. Negative scores occur when the agent takes too long or makes illegal moves.

* **Agent Performance**: An average score of -22.4 suggests that your agent, on average, is either taking too long to complete the task, making illegal moves, or both. This is indicative of a performance level that has significant room for improvement.

* **Reasons for Low Score**:
Insufficient Training: The agent may not have learned the optimal (or near-optimal) policy yet. This could be due to not training for enough episodes, having a suboptimal learning rate, or needing more exploration during the training phase.

* **Hyperparameter Tuning**: The hyperparameters (learning rate, discount factor, epsilon decay, etc.) may not be optimal for learning in this environment.


* **Improvement Strategies**:
Longer or More Efficient Training: Increase the number of training episodes or adjust the training loop to ensure more efficient learning.

* **Tune Hyperparameters**: Experiment with different values of learning rate, discount factor, epsilon start/end values, and decay rate.

* **Experiment with Network Architecture**: Try different neural network architectures to find one that learns more effectively.

* **Review Reward Structure**: Ensure that the reward structure in your environment effectively guides the agent towards the desired behavior.